<a href="https://colab.research.google.com/github/lobobranco96/pokemon-data-project/blob/main/pokemondata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


EXTRACT

In [ ]:
def fetch_pokemon_from_api():
    """
    Busca dados de Pokémon na PokeAPI, especificamente recuperando as gerações disponíveis
    e suas respectivas espécies de Pokémon.

    Retorna:
        dict: Um dicionário contendo duas listas:
            - "region": Nomes das regiões correspondentes a cada geração.
            - "pokemon_id": IDs dos Pokémon em cada geração.
    """

    def find_geracao_available():
        """
        Faz uma requisição à PokeAPI para obter todas as gerações de Pokémon disponíveis.

        Retorna:
            list: Lista com os nomes das gerações.
        """
        response = requests.get("https://pokeapi.co/api/v2/generation/", timeout=5)
        pokemon = response.json()
        return [pokemon_name['name'] for pokemon_name in pokemon.get('results')]

    def fetch_geracao(geracao):
        """
        Obtém dados de uma geração específica, incluindo a região e IDs dos Pokémon.

        Args:
            geracao (str): Nome da geração a ser buscada.

        Retorna:
            tuple: Região correspondente e lista de IDs dos Pokémon dessa geração.
        """
        response = requests.get(f"https://pokeapi.co/api/v2/generation/{geracao}", timeout=5)
        pokemon = response.json()
        regiao = pokemon.get('main_region')['name']
        ids = [nome['url'].split('/')[-2] for nome in pokemon.get('pokemon_species')]
        return regiao, ids

    # Inicializa um dicionário para armazenar os resultados das gerações
    data = {"region": [], "pokemon_id": []}
    for geracao in find_geracao_available():
        regiao, ids = fetch_geracao(geracao)
        data['region'].append(regiao)
        data['pokemon_id'].append(ids)

    return data

def fetch_poke(region, pokemon_id):
    """
    Busca dados detalhados de um Pokémon específico a partir da PokeAPI
    e retorna um dicionário contendo suas informações.

    Args:
        region (str): Região do Pokémon.
        pokemon_id (int): ID do Pokémon a ser buscado.

    Retorna:
        dict: Dicionário com informações detalhadas do Pokémon.
    """
    # Inicializa um dicionário para armazenar informações do Pokémon
    pokemon = {
        'id': None,
        'name': None,
        'region': region,
        'height': None,
        'weight': None,
        'base_experience': None,
        'types': [],
        'abilities': [],
        'moves': [],
        'location_area_encounters': [],
        'held_items': [],
        'base_happiness': None,
        'color': None,
        'capture_rate': None,
        'evolves_from': None,
        'is_baby': None,
        'is_mythical': None,
        'is_legendary': None
    }

    try:
        # Faz a requisição para obter dados do Pokémon
        response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}", timeout=5)
        response.raise_for_status()  # Lança um erro se a requisição falhar
        pokes = response.json()

        # Armazena informações básicas do Pokémon
        pokemon['id'] = pokes.get('id')
        pokemon['name'] = pokes.get('name')
        pokemon['height'] = pokes.get('height')
        pokemon['weight'] = pokes.get('weight')
        pokemon['base_experience'] = pokes.get('base_experience')
        pokemon['abilities'] = [ability['ability']['name'] for ability in pokes.get('abilities', [])]
        pokemon['moves'] = [move['move']['name'] for move in pokes.get('moves', [])]
        pokemon['held_items'] = [item['item']['name'] for item in pokes.get('held_items', [])] or 'Não encontrado.'
        pokemon['types'] = [type['type']['name'] for type in pokes.get('types', [])]

        # Armazena os stats do Pokémon
        for stat in pokes.get('stats', []):
            pokemon[stat['stat']['name']] = stat['base_stat']

        # Função interna para buscar áreas onde o Pokémon pode ser encontrado
        def fetch_pokemon_areas(id_pokemon):
            """
            Busca as áreas onde o Pokémon pode ser encontrado.

            Args:
                id_pokemon (int): ID do Pokémon a ser buscado.

            Retorna:
                list: Lista de áreas de encontro do Pokémon.
            """
            response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{id_pokemon}/encounters")
            poke_area = response.json()
            return [entry['location_area']['name'] for entry in poke_area]

        # Atualiza o dicionário com as áreas de encontro
        pokemon['location_area_encounters'] = fetch_pokemon_areas(pokemon['id'])

        # Função interna para buscar detalhes adicionais do Pokémon
        def pokemon_details(pokemon_id):
            """
            Busca detalhes adicionais de um Pokémon a partir de seu nome.

            Args:
                pokemon_name (str): Nome do Pokémon.

            Retorna:
                dict: Dicionário com detalhes adicionais do Pokémon.
            """
            response = requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}")
            pokemon_species = response.json()
            evolves_from = pokemon_species.get('evolves_from_species')
            evolves_from_name = evolves_from['name'] if evolves_from else 'Não encontrado.'

            return {
                'base_happiness': pokemon_species.get('base_happiness'),
                'color': pokemon_species.get('color', {}).get('name'),
                'capture_rate': pokemon_species.get('capture_rate'),
                'evolves_from': evolves_from_name,
                'is_baby': pokemon_species.get('is_baby'),
                'is_mythical': pokemon_species.get('is_mythical'),
                'is_legendary': pokemon_species.get('is_legendary')
            }

        # Obtém os detalhes adicionais do Pokémon
        detail = pokemon_details(pokemon_id)
        pokemon['base_happiness'] = detail['base_happiness']
        pokemon['color'] = detail['color']
        pokemon['capture_rate'] = detail['capture_rate']
        pokemon['evolves_from'] = detail['evolves_from']
        pokemon['is_baby'] = detail['is_baby']
        pokemon['is_mythical'] = detail['is_mythical']
        pokemon['is_legendary'] = detail['is_legendary']

    except requests.exceptions.RequestException as e:
        # Trata erros de requisição
        print(f"Error fetching Pokémon data (ID: {pokemon_id}, Region: {region}): {e}")
        response.raise_for_status()
    return pokemon

# Loop pelas regiões disponíveis no dicionário completo
complete_dicionario = fetch_pokemon_from_api()

for region_index in range(len(complete_dicionario['region'])):
    all_pokemon_data = []  # Lista temporária para armazenar os dados dos Pokémon
    region = complete_dicionario['region'][region_index]

    # Loop pelos IDs dos Pokémon na região atual
    for pokemon_id in complete_dicionario['pokemon_id'][region_index]:
      pokemon_data = fetch_poke(region, pokemon_id)
      all_pokemon_data.append(pokemon_data)  # Adiciona os dados à lista

    # Cria o DataFrame apenas uma vez com todos os Pokémon da região
    dataframe = pd.DataFrame(all_pokemon_data)

    diretorio = "/content/drive/MyDrive/pokemon_region/"
    nome_arquivo = f"{region}.json"
    caminho_arquivo = os.path.join(diretorio, nome_arquivo)
    dataframe.to_json(caminho_arquivo, orient='records', lines=True)
    print(f'Arquivo: {nome_arquivo} salvo com sucesso.')
